In [103]:
import pandas as pd
import time
import numpy as np
import torch
from sklearn.metrics import classification_report 
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier

In [104]:
#Import Java Data
java_dataset = pd.read_parquet('java_train-00000-of-00001.parquet')
java_df = pd.DataFrame(java_dataset)

java_dataset_test = pd.read_parquet('java_test-00000-of-00001.parquet')
java_df_test = pd.DataFrame(java_dataset_test)

java_df.head()

,index,class,comment_sentence,partition,combo,labels
0,0,Abfss.java,azure blob file system implementation of abstr...,0,azure blob file system implementation of abstr...,"[1, 0, 0, 0, 0, 0, 0]"
1,1,Abfss.java,this impl delegates to the old filesystem,0,this impl delegates to the old filesystem | Ab...,"[0, 0, 1, 0, 0, 0, 0]"
2,2,AbstractContractGetFileStatusTest.java,test getfilestatus and related listing operati...,0,test getfilestatus and related listing operati...,"[1, 0, 0, 0, 0, 0, 0]"
3,11,AbstractContractGetFileStatusTest.java,path filter which only expects paths whose fin...,0,path filter which only expects paths whose fin...,"[0, 0, 1, 0, 0, 0, 0]"
4,15,AbstractContractGetFileStatusTest.java,a filesystem filter which exposes the protecte...,0,a filesystem filter which exposes the protecte...,"[0, 0, 1, 0, 0, 0, 0]"


In [105]:
#Import Python Data
python_dataset = pd.read_parquet('python_train-00000-of-00001.parquet')
python_df = pd.DataFrame(python_dataset)

python_dataset_test = pd.read_parquet('python_test-00000-of-00001.parquet')
python_df_test = pd.DataFrame(python_dataset_test)

python_df.head()

,index,class,comment_sentence,partition,combo,labels
0,1,AccessMixin,functionality.,0,functionality. | AccessMixin,"[0, 0, 0, 0, 1]"
1,8,Atomic,"when it s used as a decorator, call wraps the ...",0,"when it s used as a decorator, call wraps the ...","[1, 0, 0, 0, 0]"
2,10,Atomic,"when it s used as a context manager, enter cre...",0,"when it s used as a context manager, enter cre...","[1, 0, 0, 0, 0]"
3,12,Atomic,exit commits the transaction or releases the s...,0,exit commits the transaction or releases the s...,"[1, 0, 0, 0, 0]"
4,14,Atomic,it s possible to disable the creation of savep...,0,it s possible to disable the creation of savep...,"[1, 0, 0, 0, 0]"


In [106]:
#Import Pharo Data
pharo_dataset = pd.read_parquet('pharo_train-00000-of-00001.parquet')
pharo_df = pd.DataFrame(pharo_dataset)

pharo_dataset_test = pd.read_parquet('pharo_test-00000-of-00001.parquet')
pharo_df_test = pd.DataFrame(pharo_dataset_test)

pharo_df.head()

,index,class,comment_sentence,partition,combo,labels
0,0,BlAnchorRelativeToElement,relative anchor takes an arbitrary element as ...,0,relative anchor takes an arbitrary element as ...,"[0, 0, 1, 0, 0, 0, 0]"
1,2,BlArrowheadSimpleArrow,i am a simple arrow like arrowhead.,0,i am a simple arrow like arrowhead. | BlArrowh...,"[0, 0, 0, 0, 1, 0, 0]"
2,3,BlArrowheadSimpleArrow,both my size and arrow length depend on the cu...,0,both my size and arrow length depend on the cu...,"[0, 0, 0, 0, 1, 0, 0]"
3,4,BlArrowheadSimpleArrow,it is possible hovewer to customize a length f...,0,it is possible hovewer to customize a length f...,"[0, 0, 0, 0, 0, 1, 0]"
4,8,BlArrowheadTriangle,i am a triangular arrow head.,0,i am a triangular arrow head. | BlArrowheadTri...,"[0, 0, 0, 0, 1, 0, 0]"


In [107]:
#Get the comment_sentence and label data columns
#Java comments and labels
java_sentences_train = java_df['comment_sentence']
java_y_train = java_df['labels']

java_sentences_test = java_df_test['comment_sentence']
java_y_test = java_df_test['labels']

java_np_sentence = java_sentences_train.to_numpy()
print(type(java_np_sentence))

#Python comments and labels
python_sentences_train = python_df['comment_sentence']
python_y_train = python_df['labels']

python_sentences_test = python_df_test['comment_sentence']
python_y_test = python_df_test['labels']

python_np_sentence = python_sentences_train.to_numpy()
print(type(python_np_sentence))

#Pharo comments and labels
pharo_sentences_train = pharo_df['comment_sentence']
pharo_y_train = pharo_df['labels']

pharo_sentences_test = pharo_df_test['comment_sentence']
pharo_y_test = pharo_df_test['labels']

pharo_np_sentence = pharo_sentences_train.to_numpy()
print(type(pharo_np_sentence))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [ ]:
#convert label classes to single integer labels so the model can predict different classes(0-6 since the largest # of classes is 7) 
java_y_label = []
java_y_tlabel = []

python_y_label = []
python_y_tlabel = []

pharo_y_label = []
pharo_y_tlabel = []


def conv_cl_to_label(y_class):
    y_label = []
    for arr in y_class:
        if arr[0] == 1:
            y_label.append(0)
        elif arr[1] == 1:
            y_label.append(1)
        elif arr[2] == 1:
            y_label.append(2)
        elif arr[3] == 1:
            y_label.append(3)
        elif arr[4] == 1:
            y_label.append(4)
        elif arr[5] == 1:
            y_label.append(5)
        elif arr[6] == 1:
            y_label.append(6)
    return y_label

#{lang}_y_label represents the numeric version of the class associated with each comment sentence in the training dataset
#{lang}_y_tlabel represents the numeric version of the class associated with each comment sentence in the test dataset
java_y_label = conv_cl_to_label(java_y_train)
java_y_tlabel = conv_cl_to_label(java_y_test)

python_y_label = conv_cl_to_label(python_y_train)
python_y_tlabel = conv_cl_to_label(python_y_test)

pharo_y_label = conv_cl_to_label(pharo_y_train)
pharo_y_tlabel = conv_cl_to_label(pharo_y_test)

#length of the train and test datasets
print(len(java_y_label))
print(len(java_y_tlabel))

print(len(python_y_label))
print(len(python_y_tlabel))

print(len(pharo_y_label))
print(len(pharo_y_tlabel))



7614
1725
1884
406
1298
289


In [ ]:
#Convert the imported sentences to numpy arrays and flatten so the input is compatible with the input layer into the neural network
java_npt_sentence = java_sentences_test.to_numpy()
java_npf_sentence = java_np_sentence.flatten()
java_nptf_sentence = java_npt_sentence.flatten()

python_npt_sentence = python_sentences_test.to_numpy()
python_npf_sentence = python_np_sentence.flatten()
python_nptf_sentence = python_npt_sentence.flatten()

pharo_npt_sentence = pharo_sentences_test.to_numpy()
pharo_npf_sentence = pharo_np_sentence.flatten()
pharo_nptf_sentence = pharo_npt_sentence.flatten()

#padding the comments so each sentence is 40 tokens long
#if comment is > 40 then discards remaining values
#if comment is < 40 then pads using a custom <PAD> token 
max_len = 40

def padding(npt_sentence):
    for sentence in npt_sentence:
        if len(sentence) > max_len:
            sentence =sentence[:max_len]
        else:
            while(max_len> len(sentence)):
                sentence = sentence + "<PAD>"
    return npt_sentence


java_npf_sentence = padding(java_npf_sentence)
java_nptf_sentence = padding(java_nptf_sentence)

python_npf_sentence = padding(python_npf_sentence)
python_nptf_sentence = padding(python_nptf_sentence)

pharo_npf_sentence = padding(pharo_npf_sentence)
pharo_nptf_sentence = padding(pharo_nptf_sentence)


#Creating a scikit pipeline for both text vectorization, tokenization, and model call
#See 'Machine learning with pytorch and sci-kit' pg. 174/175 for explanation of Pipelines
#The below pipeline takes in the padded code comments and tokenizes them so each word is represented as a number. This is then passed into the MLP classifier which is a custoim dense neural network
#with 6 layers (1 input, 5 hidden) 
vect_model = Pipeline([
    
    ('token_vect', TfidfVectorizer()),
    ('nn', MLPClassifier(hidden_layer_sizes=(2000,1028,512,256,512,64), activation='relu', solver='adam', alpha=0.0001, batch_size='auto', learning_rate='constant', learning_rate_init=0.001, max_iter=500)),
    
])


In [110]:
# Train the model for each language on its respective dataset
java_model = vect_model.fit(java_npf_sentence, java_y_label)

python_model = vect_model.fit(python_npf_sentence, python_y_label)

pharo_model = vect_model.fit(pharo_npf_sentence, pharo_y_label)



In [111]:
#Run the model 10 times to get an average prediction time per NLBSE requirements

def predictions(model, test_sentences):
    total_time = 0
    begin = time.time()
    for i in range(10):
        predictions = model.predict(test_sentences)
        total = time.time() - begin
        total_time = total_time + total
    return total_time, predictions

java_total_time, java_predictions = predictions(java_model, java_nptf_sentence)
python_total_time, python_predictions = predictions(python_model, python_nptf_sentence)
pharo_total_time, pharo_predictions = predictions(pharo_model, pharo_nptf_sentence)
#Total cumulative run time for all models
total_time = java_total_time + python_total_time + pharo_total_time

In [112]:
#Calculate avg overall stats
def avg_f1(java_f1, python_f1, pharoh_f1):
    return (java_f1 + python_f1 + pharoh_f1)/3

#Get precision, recall, and f1 average for test dataset for java, python, and pharoh from scikits classification report
#java stats
print("Java comment classification results:")
print(classification_report(java_y_tlabel, java_predictions))

print("Python comment classification results:")
print(classification_report(python_y_tlabel, python_predictions))

print("Pharo comment classification results:")
print(classification_report(pharo_y_tlabel, pharo_predictions))

#using the classification_report function the avg f1, precision, and recall can be determined for each model. 'macro avg' refers to the avg over the entire dataset
java_stats = classification_report(java_y_tlabel, java_predictions,output_dict=True)

java_precision_avg =  java_stats['macro avg']['precision'] 
java_recall_avg = java_stats['macro avg']['recall']    
java_f1_avg = java_stats['macro avg']['f1-score']

#python stats
python_stats = classification_report(python_y_tlabel, python_predictions,output_dict=True)

python_precision_avg =  python_stats['macro avg']['precision'] 
python_recall_avg = python_stats['macro avg']['recall']    
python_f1_avg = python_stats['macro avg']['f1-score']

#pharoh stats
pharoh_stats = classification_report(pharo_y_tlabel, pharo_predictions,output_dict=True)

pharoh_precision_avg =  pharoh_stats['macro avg']['precision'] 
pharoh_recall_avg = pharoh_stats['macro avg']['recall']    
pharoh_f1_avg = pharoh_stats['macro avg']['f1-score']

Java comment classification results:
              precision    recall  f1-score   support

           0       0.56      0.24      0.34       892
           1       0.05      0.80      0.10        45
           2       0.06      0.25      0.09       100
           3       0.30      0.01      0.01       427
           4       0.17      0.17      0.17       178
           5       0.02      0.07      0.03        15
           6       0.00      0.00      0.00        68

    accuracy                           0.18      1725
   macro avg       0.17      0.22      0.11      1725
weighted avg       0.38      0.18      0.20      1725

Python comment classification results:
              precision    recall  f1-score   support

           0       0.26      0.22      0.24       121
           1       0.32      0.39      0.35       127
           2       0.09      0.27      0.14        33
           3       0.00      0.00      0.00        48
           4       0.35      0.16      0.22        77
  

/home/jvanake/11_environment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jvanake/11_environment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jvanake/11_environment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jvanake/11_en

In [113]:
java_flops = 0
python_flops = 0
pharo_flops = 0

#torch.profiler.profile(with_flops=True) does not work with scikit learn pipelines so an equivalent estimate for flops is used
#per nvidia (https://docs.nvidia.com/deeplearning/performance/dl-performance-gpu-background/index.html#gpu-arch) a flop is 2 * the Input/Output of a dense layer. For example the java model contains FLOPS = (1725*2000 + 2000 * 1028 + 1028 * 512 + 512 * 256 + 256 * 512 + 512 *64) * 2 = 12,654,496 = .01265 GFLOPS 
def FLOPS(input_size):
    return ((input_size*2000 + 2000 * 1028 + 1028 * 512 + 512 * 256 + 256 * 512 + 512 *64) * 2)/1e9


java_flops = FLOPS(len(java_nptf_sentence))
python_flops = FLOPS(len(python_nptf_sentence))
pharo_flops = FLOPS(len(pharo_nptf_sentence))

#combine the total flops
total_flops = java_flops + python_flops + pharo_flops


In [114]:
print("Compute in GFLOPs:", total_flops)
print("Avg runtime in seconds:", total_time/10)

Compute in GFLOPs: 0.026943488
Avg runtime in seconds: 0.42482657432556153


In [115]:
#NLBSE25 score determination

max_avg_runtime = 5
max_avg_flops = 5000

# s𝑢𝑏𝑚𝑖𝑠𝑠𝑖𝑜𝑛_𝑠𝑐𝑜𝑟𝑒(𝑚𝑜𝑑𝑒𝑙)=(𝑎𝑣𝑔. 𝐹1)×0.60+((𝑚𝑎𝑥_𝑎𝑣𝑔_𝑟𝑢𝑛𝑡𝑖𝑚𝑒−𝑚𝑒𝑎𝑠𝑢𝑟𝑒𝑑_𝑎𝑣𝑔_𝑟𝑢𝑛𝑡𝑖𝑚𝑒)/𝑚𝑎𝑥_𝑎𝑣𝑔_𝑟𝑢𝑛𝑡𝑖𝑚𝑒)×0.2+((𝑚𝑎𝑥_GFLOPs−𝑚𝑒𝑎𝑠𝑢𝑟𝑒𝑑_GFLOPs)/𝑚𝑎𝑥_GFLOPs)×0.2
def score(avg_f1, avg_runtime, avg_flops):
    return (0.6 * avg_f1 +
      0.2 * ((max_avg_runtime - avg_runtime) / max_avg_runtime) +
      0.2 * ((max_avg_flops - avg_flops) / max_avg_flops))


avg_runtime = total_time/10
avg_flops = total_flops/10



round(score(avg_f1(java_f1_avg, python_f1_avg, pharoh_f1_avg), avg_runtime, avg_flops), 2)

0.5